In [1]:
from zipline.api import order, record, symbol, set_benchmark
from zipline.data.resample import minute_panel_to_session_panel as mp
from datetime import datetime
import zipline
import pickle
from pandas import Timestamp
import pandas as pd
from collections import OrderedDict
import pytz
from trading_calendars.exchange_calendar_nyse import NYSEExchangeCalendar

In [2]:
def initialize(context):
    context.i = 0
    context.assets = symbol('EURUSD')
    
def handle_data(context, data):
    context.i+=1
    if context.i<200:
        return
    window = data.history(context.assets, ticker, 200, "1m")
    ema_200 = pd.ewma(window, 200, 199)
    ema_50 = pd.ewma(window, 50, 49)
    ema_20 = pd.ewma(window, 20, 19)
    ema_10 = pd.ewma(window, 10, 9)
    print(ema_10)
    
    order(symbol("EURUSD"), 10)
    record(SPY=data.current(symbol('EURUSD'), 'price'))

df = pd.read_csv('Stock Market/sp500_joined_closes.csv')
df['Date'] = pd.to_datetime(df['Date'].apply(str))
df.set_index('Date',inplace=True)

with open('Stock Market/sp500tickers.pickle', 'rb') as f:
    tickers = pickle.load(f)
print(df.head())
data = OrderedDict()
for ticker in tickers:
    data[ticker] = df
panel = pd.Panel(data)
panel.minor_axis = tickers
panel.major_axis = panel.major_axis.tz_localize(pytz.utc)

perf = zipline.run_algorithm(start=datetime(2017, 1, 2, 0, 0, 0, 0, pytz.utc),
                              end=datetime(2017, 12, 29, 0, 0, 0, 0, pytz.utc),
                              initialize=initialize,
                              capital_base=100000,
                              trading_calendar=NYSEExchangeCalendar(),
                              handle_data=handle_data,
                              data_frequency='daily',
                              data=panel)


                  MMM        ABT  ABBV    ABMD  ACN      ATVI       ADBE  \
Date                                                                       
2000-01-03  30.790071  10.071733   NaN  18.250  NaN  1.252334  16.277607   
2000-01-04  29.566689   9.783969   NaN  17.815  NaN  1.214246  14.912036   
2000-01-05  31.073895   9.729294   NaN  18.065  NaN  1.218816  15.177701   
2000-01-06  32.884500  10.109142   NaN  18.030  NaN  1.195202  15.331638   
2000-01-07  33.536970  10.215615   NaN  17.940  NaN  1.227958  16.076495   

               AMD  AAP        AES ...   WYNN       XEL        XRX       XLNX  \
Date                               ...                                          
2000-01-03  15.500  NaN  27.856077 ...    NaN  8.469647  74.282588  34.902729   
2000-01-04  14.625  NaN  26.753361 ...    NaN  8.665786  70.812587  34.111352   
2000-01-05  14.655  NaN  26.918576 ...    NaN  8.888671  71.211791  33.409564   
2000-01-06  16.000  NaN  27.256691 ...    NaN  8.915418  72.93

KeyboardInterrupt: 